In [1]:
import tensorflow as tf

In [2]:
import csv
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import imblearn
import warnings
warnings.filterwarnings('ignore')

In [3]:
train_df = pd.read_csv('econet dataset/train.csv')
unique_locations = train_df['Station'].unique()

In [4]:
train_df['Ob'] = pd.to_datetime(train_df['Ob'])

In [5]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
label_encoder = LabelEncoder()
features = ['Station', 'Ob', 'value', 'measure', 'R_flag', 'I_flag', 'Z_flag', 'B_flag']
temp_df = train_df.loc[:]
label_encoder = LabelEncoder()
temp_df['Station'] = label_encoder.fit_transform(train_df['Station'])
temp_df['measure'] = label_encoder.fit_transform(train_df['measure'])
temp_df['Ob'] = train_df['Ob'].values.astype(float)

x = temp_df.loc[:, features].values
y = temp_df.loc[:,['target']].values
x = StandardScaler().fit_transform(x)

In [6]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.99)
principalComponents = pca.fit_transform(x)
p_df = pd.DataFrame(principalComponents)

In [7]:
finalDf = pd.concat([p_df, temp_df[['target']]], axis = 1)
print(finalDf.head())

          0         1         2         3         4         5         6  \
0 -0.326512  0.946291  0.364007  2.680728 -0.883147  0.672595 -0.590456   
1 -0.325359  0.945896  0.362786  2.679443 -0.883103  0.673863 -0.591416   
2 -0.325825  0.946202  0.362537  2.679315 -0.883461  0.673131 -0.590933   
3 -0.327426  0.947174  0.362091  2.679230 -0.884507  0.670732 -0.589325   
4 -0.289066  0.986795  0.054464  2.403466 -1.005693  0.633474 -0.586918   

          7  target  
0  0.775774   False  
1  0.775882   False  
2  0.775960   False  
3  0.776162   False  
4  0.823686   False  


In [8]:
from scipy import stats

def create_windows(X, y, time_steps=1, step=1):
    Xs, ys = [], []
      
    for i in range(0, len(X) - time_steps + 1, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)
        ys.append(stats.mode(labels)[0][0])

    return np.array(Xs), np.array(ys).reshape(-1, 1)

In [9]:
# y = label_encoder.fit_transform(finalDf['target'])
y = finalDf['target']
X = finalDf[[0, 1, 2, 3, 4, 5, 6, 7]]
print(type(y), type(X))

<class 'pandas.core.series.Series'> <class 'pandas.core.frame.DataFrame'>


In [10]:
print(X.shape, y.shape)

(6593274, 8) (6593274,)


In [11]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

under = RandomUnderSampler(random_state = 42)

X, y = under.fit_resample(X, y)
print(X.shape, y.shape)

(470344, 8) (470344,)


In [12]:
pd.DataFrame(y).value_counts()

target
False     235172
True      235172
dtype: int64

In [13]:
TIME_STEPS = 40
STEP = 1

X, y = create_windows(X, y, time_steps=TIME_STEPS,step=STEP)

In [39]:
y.shape

(470305, 1)

In [14]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
enc = enc.fit(y_train)
y_train = enc.transform(y_train)
y_test = enc.transform(y_test)

In [15]:
y_test[0]

array([0., 1.])

In [16]:
y_train.shape

(376244, 2)

In [17]:
print(X_train.shape, y_train.shape)

(376244, 40, 8) (376244, 2)


In [18]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam, SGD

input_shape = (X_train.shape[1], X_train.shape[2])
print(input_shape)
output_dim = 2

model=Sequential()
model.add(LSTM(4,return_sequences=True,input_shape=input_shape))
model.add(Dropout(0.1)) 
model.add(LSTM(4, return_sequences=False))
model.add(Dropout(0.1))
model.add(Dense(output_dim))
model.add(Activation("softmax"))
model.compile(loss='categorical_crossentropy',optimizer='adam')

(40, 8)


In [19]:
hist = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=64,
    shuffle=True
)

Epoch 1/10
5879/5879 [==============================] - 59s 9ms/step - loss: 0.0372
Epoch 2/10
5879/5879 [==============================] - 61s 10ms/step - loss: 0.0015
Epoch 3/10
5879/5879 [==============================] - 61s 10ms/step - loss: 6.8476e-04
Epoch 4/10
5879/5879 [==============================] - 53s 9ms/step - loss: 5.1427e-04
Epoch 5/10
5879/5879 [==============================] - 58s 10ms/step - loss: 2.5628e-04
Epoch 6/10
5879/5879 [==============================] - 57s 10ms/step - loss: 2.4667e-04
Epoch 7/10
5879/5879 [==============================] - 53s 9ms/step - loss: 3.9925e-04
Epoch 8/10
5879/5879 [==============================] - 54s 9ms/step - loss: 1.7007e-04
Epoch 9/10
5879/5879 [==============================] - 52s 9ms/step - loss: 3.1228e-04
Epoch 10/10
5879/5879 [==============================] - 52s 9ms/step - loss: 1.7221e-04


In [20]:
import pickle
filename = 'lstm_model.pkl'
with open(filename, 'wb') as f:
    pickle.dump(hist,  f)

INFO:tensorflow:Assets written to: ram://49c63bc8-0cd9-48b0-94f5-c6d880cb453b/assets


INFO:tensorflow:Assets written to: ram://49c63bc8-0cd9-48b0-94f5-c6d880cb453b/assets


In [21]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis = 1)
y_test = np.argmax(y_test, axis = 1)

In [22]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     46832
           1       1.00      1.00      1.00     47229

    accuracy                           1.00     94061
   macro avg       1.00      1.00      1.00     94061
weighted avg       1.00      1.00      1.00     94061



In [23]:
import pandas as pd
print(pd.DataFrame(y_pred).value_counts())

1    47229
0    46832
dtype: int64


In [24]:
print(pd.DataFrame(y_test).value_counts())

1    47229
0    46832
dtype: int64


In [25]:
test_df = pd.read_csv('econet dataset/test.csv')

In [26]:
test_df.shape

(1856106, 8)

In [27]:
test_df['Ob'] = pd.to_datetime(test_df['Ob'])

In [28]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

features = ['Station', 'Ob', 'value', 'measure', 'R_flag', 'I_flag', 'Z_flag', 'B_flag']
temp_df = test_df.loc[:]

label_encoder = LabelEncoder()
temp_df['Station'] = label_encoder.fit_transform(test_df['Station'])
temp_df['measure'] = label_encoder.fit_transform(test_df['measure'])
temp_df['Ob'] = test_df['Ob'].values.astype(float)

x = temp_df.loc[:, features].values
x = StandardScaler().fit_transform(x)

In [29]:
x.shape

(1856106, 8)

In [30]:
def test_windows(X, time_steps, step):
    Xs = []
    n_records = X.shape[0]
    remainder = (n_records - time_steps) % step
    num_windows = 1 + int((n_records - time_steps - remainder) / step)
    for k in range(num_windows):
        Xs.append(X[k*step:time_steps-1+k*step+1])
    return np.array(Xs)

In [31]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.99)
principalComponents = pca.fit_transform(x)
p_df = pd.DataFrame(principalComponents)

In [32]:
p_df.shape

(1856106, 8)

In [33]:
finalDf = test_windows(p_df, time_steps=TIME_STEPS,step=STEP)

In [34]:
finalDf.shape

(1856067, 40, 8)

In [35]:
y_preds_test = model.predict(finalDf)

In [36]:
pred_test = np.concatenate([y_preds_test, np.asarray([y_preds_test[-1]]*39)])

In [37]:
print(pred_test.shape)

(1856106, 2)


In [38]:
pd.DataFrame(pred_test[:,1], columns=['target']).to_csv('predictions.csv', index=False)